In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE94104"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE94104"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE94104.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE94104.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE94104.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional analysis of locally advanced rectal cancer pre-therapeutic biopsies and post-therapeutic resections"
!Series_summary	"Understanding transcriptional changes in locally advanced rectal cancer which are therapy-related and dependent upon tumour regression will drive stratified medicine in the rectal cancer paradigm"
!Series_overall_design	"Total RNA was obtained from 40 matched formalin fixed paraffin embedded (FFPE) LARC biopsy and resections specimens provided by the Northern Ireland Biobank and arrayed using the Illumina HumanHT-12 WG-DASL V4 expression beadchip"
Sample Characteristics Dictionary:
{0: ['tissue: Locally Advanced Rectal Cancer (LARC)'], 1: ['tissue type: Biopsy', 'tissue type: Resection'], 2: ['tumour regression grade: 1', 'tumour regression grade: 2', 'tumour regression grade: 3']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Availability
# Based on the background information, the dataset appears to contain gene expression data
# as it mentions "expression beadchip" data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait: "tumour regression grade" is recorded in row 2
trait_row = 2

# Age is not explicitly mentioned in the characteristics dictionary
age_row = None

# Gender is not explicitly mentioned in the characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert tumour regression grade to binary.
    Grade 1-2 (good regression) -> 0, Grade 3 (poor regression) -> 1
    """
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        grade = int(value)
        if grade == 1 or grade == 2:
            return 0  # Good regression
        elif grade == 3:
            return 1  # Poor regression
        else:
            return None
    except:
        return None

def convert_age(value):
    """Placeholder function for age conversion, not used in this dataset"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion, not used in this dataset"""
    return None

# 3. Save Metadata
# Determine trait availability based on whether trait_row is None
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting DataFrame
    print("Clinical features preview:")
    print(preview_df(clinical_features))
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)


Clinical features preview:
{'GSM2469019': [0.0], 'GSM2469020': [0.0], 'GSM2469021': [0.0], 'GSM2469022': [0.0], 'GSM2469023': [0.0], 'GSM2469024': [0.0], 'GSM2469025': [0.0], 'GSM2469026': [0.0], 'GSM2469027': [1.0], 'GSM2469028': [1.0], 'GSM2469029': [0.0], 'GSM2469030': [0.0], 'GSM2469031': [1.0], 'GSM2469032': [1.0], 'GSM2469033': [1.0], 'GSM2469034': [1.0], 'GSM2469035': [0.0], 'GSM2469036': [0.0], 'GSM2469037': [0.0], 'GSM2469038': [0.0], 'GSM2469039': [1.0], 'GSM2469040': [1.0], 'GSM2469041': [0.0], 'GSM2469042': [0.0], 'GSM2469043': [0.0], 'GSM2469044': [0.0], 'GSM2469045': [0.0], 'GSM2469046': [0.0], 'GSM2469047': [0.0], 'GSM2469048': [0.0], 'GSM2469049': [0.0], 'GSM2469050': [0.0], 'GSM2469051': [0.0], 'GSM2469052': [0.0], 'GSM2469053': [0.0], 'GSM2469054': [0.0], 'GSM2469055': [0.0], 'GSM2469056': [0.0], 'GSM2469057': [0.0], 'GSM2469058': [0.0], 'GSM2469059': [0.0], 'GSM2469060': [0.0], 'GSM2469061': [1.0], 'GSM2469062': [1.0], 'GSM2469063': [1.0], 'GSM2469064': [1.0], 'GSM24

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are Illumina probe IDs (ILMN_*), which are not human gene symbols
# They need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Addres

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# 'ID' in gene_annotation corresponds to probe IDs (ILMN_*) in gene_data
# 'Symbol' appears to contain gene symbols to map to
probe_col = 'ID'  # Column containing probe identifiers
gene_col = 'Symbol'  # Column containing gene symbols

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Found mapping for {len(mapping_df)} probes")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted expression data to {len(gene_data)} genes")
print("First few genes after mapping:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)


Found mapping for 29377 probes
First few rows of mapping dataframe:
             ID        Gene
0  ILMN_3166687  ERCC-00162
1  ILMN_3165566  ERCC-00071
2  ILMN_3164811  ERCC-00009
3  ILMN_3165363  ERCC-00053
4  ILMN_3166511  ERCC-00144
Converted expression data to 18407 genes
First few genes after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT',
       'A4GNT', 'AAA1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row, 
    convert_age=convert_age,
    gender_row=gender_row, 
    convert_gender=convert_gender
)

# Save the clinical features data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Clinical data saved to ../../output/preprocess/Rectal_Cancer/clinical_data/GSE94104.csv
Normalized gene data shape: (17833, 80)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE94104.csv
Linked data shape: (80, 17834)
            Rectal_Cancer       A1BG  A1BG-AS1       A1CF        A2M  \
GSM2469019            0.0   7.395705  8.398890  28.693917  13.337807   
GSM2469020            0.0   8.551503  8.688974  26.624397  14.148331   
GSM2469021            0.0  10.632415  7.824079  23.596701  13.451809   
GSM2469022            0.0   8.816704  7.720825  25.802108  13.616095   
GSM2469023            0.0   9.020842  7.200367  30.043000  13.611848   

               A2ML1     A4GALT     A4GNT       AAA1       AAAS  ...  \
GSM2469019  7.472346  12.487900  5.906503  37.780392  10.477635  ...   
GSM2469020  8.259388  12.611664  5.611697  31.996606  11.295834  ...   
GSM2469021  8.393553  10.865053  5.687393  40.876755   9.940273  ...   
GSM2469022  5.889630  11.647056  5.497962  36.785756  10.388437  ...   
GSM2469023  6.835435   9.823508  5.313831  48.428284   8.857288  ...   

            

Shape after handling missing values: (80, 17834)
For the feature 'Rectal_Cancer', the least common label is '1.0' with 22 occurrences. This represents 27.50% of the dataset.
The distribution of the feature 'Rectal_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Rectal_Cancer/GSE94104.csv
